This notebook replicates the Airflow DAG and its tasks for stock price prediction using a Random Forest model.

In [25]:

!pip uninstall -y numpy pandas scipy scikit-learn joblib

!pip install numpy pandas scipy scikit-learn joblib

!pip install pandas-gbq google-auth holidays apache-airflow-providers-google



from datetime import datetime, timedelta, date
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import os
import logging
import holidays

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: pandas 2.1.4
Uninstalling pandas-2.1.4:
  Successfully uninstalled pandas-2.1.4
Found existing installation: scipy 1.15.2
Uninstalling scipy-1.15.2:
  Successfully uninstalled scipy-1.15.2
Found existing installation: scikit-learn 1.5.2
Uninstalling scikit-learn-1.5.2:
  Successfully uninstalled scikit-learn-1.5.2
Found existing installation: joblib 1.4.2
Uninstalling joblib-1.4.2:
  Successfully uninstalled joblib-1.4.2
Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.2.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Using cached pandas-2.2.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
  Using cached scipy-1.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.6 MB)
  Using cached scikit_learn-1.6.1-cp310-cp310-manylinux_2_17_x86_64.m

In [27]:
import numpy as np, pandas as pd, scipy, sklearn
from sklearn.model_selection import train_test_split
print(np.__version__, pd.__version__, scipy.__version__, sklearn.__version__)


1.26.4 2.1.4 1.15.2 1.5.2


## Default Arguments and Constants

In [28]:

PROJECT_ID = 'terraform-457118'
DATASET_ID = 'stock_market_data'
LOCATION = 'US'
TABLE_ID = 'stock_data_raw'
SERVICE_ACCOUNT_PATH = '/home/jtsarmento/Projects/airflow-dbt-demo/keys/service-account.json'
HISTORY_DAYS = '365'

# Define dataset paths
RAW_TABLE = f"`{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`"
METRICS_TABLE = f"{DATASET_ID}.model_metrics"
PREDICTIONS_TABLE = f"{DATASET_ID}.price_predictions"
MARTS_DATASET = f"{DATASET_ID}_marts"
STAGING_DATASET = f"{DATASET_ID}_staging"

In [29]:
from google.oauth2 import service_account
import pandas_gbq

creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_PATH,
    scopes=["https://www.googleapis.com/auth/bigquery"],
)

query = f"""
SELECT 
  symbol, date, close,
  LAG(close,1)  OVER(PARTITION BY symbol ORDER BY date) AS prev_close_1,
  volume,
  LAG(volume,1) OVER(PARTITION BY symbol ORDER BY date) AS prev_volume_1
FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
WHERE date >= DATE_SUB(CURRENT_DATE(), INTERVAL {HISTORY_DAYS} DAY)
ORDER BY symbol, date
"""

df_raw = pandas_gbq.read_gbq(query, project_id=PROJECT_ID, credentials=creds)
df_raw.head(), df_raw.info()

Downloading: 100%|██████████|
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1750 entries, 0 to 1749
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   symbol         1750 non-null   object 
 1   date           1750 non-null   dbdate 
 2   close          1750 non-null   float64
 3   prev_close_1   1743 non-null   float64
 4   volume         1750 non-null   Int64  
 5   prev_volume_1  1743 non-null   Int64  
dtypes: Int64(2), dbdate(1), float64(2), object(1)
memory usage: 85.6+ KB


(  symbol        date       close  prev_close_1    volume  prev_volume_1
 0   AAPL  2024-04-26  168.504349           NaN  44838400           <NA>
 1   AAPL  2024-04-29  172.684601    168.504349  68169400       44838400
 2   AAPL  2024-04-30  169.529495    172.684601  65934800       68169400
 3   AAPL  2024-05-01  168.504349    169.529495  50383100       65934800
 4   AAPL  2024-05-02  172.216827    168.504349  94214900       50383100,
 None)

## prepare_features Task

In [30]:

query = f"""
SELECT 
    symbol,
    date,
    close,
    LAG(close, 1) OVER(PARTITION BY symbol ORDER BY date) as prev_close_1,
    LAG(close, 2) OVER(PARTITION BY symbol ORDER BY date) as prev_close_2,
    LAG(close, 3) OVER(PARTITION BY symbol ORDER BY date) as prev_close_3,
    LAG(close, 4) OVER(PARTITION BY symbol ORDER BY date) as prev_close_4,
    LAG(close, 5) OVER(PARTITION BY symbol ORDER BY date) as prev_close_5,
    volume,
    LAG(volume, 1) OVER(PARTITION BY symbol ORDER BY date) as prev_volume_1
FROM 
    {RAW_TABLE}
WHERE 
    date >= DATE_SUB(CURRENT_DATE(), INTERVAL {HISTORY_DAYS} DAY)
ORDER BY 
    symbol, date
"""
from google.oauth2 import service_account
import pandas_gbq

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_PATH,
    scopes=["https://www.googleapis.com/auth/bigquery"]
)

df = pandas_gbq.read_gbq(query, project_id=PROJECT_ID, credentials=credentials)
df = df.dropna()
df['price_change'] = df['close'] - df['prev_close_1']
df['price_change_pct'] = df['price_change'] / df['prev_close_1']
df['volume_change_pct'] = (df['volume'] - df['prev_volume_1']) / df['prev_volume_1']
# df.to_csv('/downloads/prepared_features.csv', index=False)
print(f"Prepared features for {len(df)} rows across {df['symbol'].nunique()} symbols")
print(df.head())





Downloading: 100%|██████████|
Prepared features for 1715 rows across 7 symbols
  symbol        date       close  prev_close_1  prev_close_2  prev_close_3  \
5   AAPL  2024-05-03  182.518188    172.216827    168.504349    169.529495   
6   AAPL  2024-05-06  180.856033    182.518188    172.216827    168.504349   
7   AAPL  2024-05-07  181.542786    180.856033    182.518188    172.216827   
8   AAPL  2024-05-08  181.881195    181.542786    180.856033    182.518188   
9   AAPL  2024-05-09  183.702591    181.881195    181.542786    180.856033   

   prev_close_4  prev_close_5     volume  prev_volume_1  price_change  \
5    172.684601    168.504349  163224100       94214900     10.301361   
6    169.529495    172.684601   78569700      163224100     -1.662155   
7    168.504349    169.529495   77305800       78569700      0.686752   
8    172.216827    168.504349   45057100       77305800      0.338409   
9    182.518188    172.216827   48983000       45057100      1.821396   

   price_chan

## train_model Task

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib


# Build a records list including date, features, actual, predicted, error
models = {}
records = []
features = [
    'prev_close_1','prev_close_2','prev_close_3',
    'prev_close_4','prev_close_5','prev_volume_1',
    'price_change','price_change_pct','volume_change_pct'
]

for symbol in df['symbol'].unique():
    # Prepare symbol-specific DataFrame with date
    symbol_df = df[df['symbol'] == symbol].copy()
    data = symbol_df[['date'] + features + ['close']]

    # Split
    train, test = train_test_split(data, test_size=0.2, random_state=42)

    # Train
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(train[features], train['close'])
    models[symbol] = model
    # Predict
    preds = model.predict(test[features])
    test = test.reset_index(drop=True)

    # Collect results
    for i, row in test.iterrows():
        actual = row['close']
        pred = preds[i]
        rec = {
            'symbol': symbol,
            'date': row['date'],
            'actual': actual,
            'predicted': pred,
            'error': pred - actual
        }
        # Add feature columns
        for feat in features:
            rec[feat] = row[feat]
        records.append(rec)

# Create DataFrame and display
results_df = pd.DataFrame(records)


# Sort by date ascending (and symbol if you want symbol groups)
results_df = results_df.sort_values(by=['symbol','date'], ascending=[True, True]).reset_index(drop=True)


print(results_df.head())

  symbol        date      actual   predicted     error  prev_close_1  \
0   AAPL  2024-05-13  185.656021  187.059660  1.403639    182.436829   
1   AAPL  2024-05-16  189.204102  188.059497 -1.144604    189.084503   
2   AAPL  2024-05-17  189.234009  188.783204 -0.450805    189.204102   
3   AAPL  2024-05-24  189.343628  190.296030  0.952402    186.254013   
4   AAPL  2024-05-28  189.353607  189.159872 -0.193735    189.343628   

   prev_close_2  prev_close_3  prev_close_4  prev_close_5  prev_volume_1  \
0    183.702591    181.881195    181.542786    180.856033       50759500   
1    186.802170    185.656021    182.436829    183.702591       70400000   
2    189.084503    186.802170    185.656021    182.436829       52845200   
3    190.260559    191.705704    190.400085    189.234009       51005900   
4    186.254013    190.260559    191.705704    190.400085       36294600   

   price_change  price_change_pct  volume_change_pct  
0      3.219193          0.017646           0.419336  


In [37]:
import numpy as np
from sklearn.metrics import r2_score


# 1) Compute summary metrics per symbol
summary = results_df.groupby('symbol').apply(
    lambda grp: pd.Series({
        'count':        len(grp),
        'MAE':          grp['error'].abs().mean(),
        'MSE':          (grp['error']**2).mean(),
        'RMSE':         np.sqrt((grp['error']**2).mean()),
        'Mean Error':   grp['error'].mean(),
        'R2 Score':     r2_score(grp['actual'], grp['predicted'])
    })
).reset_index()

# 2) Rank by RMSE (lowest is best)
summary = summary.sort_values('RMSE').reset_index(drop=True)

# 3) Show the ranking
print("Performance ranking by RMSE:")
print(summary)

# Optional: merge back into results_df if you want symbol-level stats on every row
results_df = results_df.merge(summary[['symbol','MAE','RMSE','R2 Score']], on='symbol', how='left')

Performance ranking by RMSE:
  symbol  count       MAE        MSE      RMSE  Mean Error  R2 Score
0     EU   49.0  0.086571   0.023766  0.154163    0.016673  0.972733
1   AMZN   49.0  1.318895   3.066013  1.751003   -0.224129  0.991356
2   AAPL   49.0  1.340316   3.412865  1.847394    0.096030  0.989881
3  GOOGL   49.0  1.105925   3.757963  1.938547   -0.253494  0.966617
4   CELH   49.0  1.288527   6.211452  2.492278    0.213421  0.986641
5   MSFT   49.0  2.116950   8.427652  2.903042    0.295749  0.974532
6   META   49.0  5.867241  56.881632  7.541991    1.317177  0.989754


In [39]:

from datetime import timedelta
import holidays

# (Re-define or copy in your trading‐day logic)
def is_trading_day(d):
    if d.weekday() >= 5:  # Saturday=5, Sunday=6
        return False
    us_hols = holidays.US(years=d.year)
    market_hols = {
        "New Year's Day", "Martin Luther King Jr. Day", "Presidents Day",
        "Good Friday", "Memorial Day", "Juneteenth National Independence Day",
        "Independence Day", "Labor Day", "Thanksgiving", "Christmas Day"
    }
    name = us_hols.get(d)
    return not (name and any(h in name for h in market_hols))

def next_trading_day(d):
    nd = d + timedelta(days=1)
    while not is_trading_day(nd):
        nd += timedelta(days=1)
    return nd

# Build predictions
next_preds = []
for symbol, model in models.items():
    sym_df = df[df['symbol']==symbol].sort_values('date')
    last_row = sym_df.iloc[-1]
    
    # feature vector for the most recent date
    X_new = last_row[features].values.reshape(1, -1)
    pred_price = model.predict(X_new)[0]
    
    # compute next trading date
    last_date = pd.to_datetime(last_row['date']).date()
    next_date = next_trading_day(last_date)
    
    next_preds.append({
        'symbol': symbol,
        'last_date':  last_date,
        'next_date':  next_date,
        'predicted_close': pred_price,
        'last_close':      last_row['close'],
        'predicted_change': pred_price - last_row['close'],
        'predicted_pct':   (pred_price - last_row['close'])/last_row['close']*100
    })

# Show as DataFrame
next_pred_df = pd.DataFrame(next_preds)
next_pred_df


/home/jtsarmento/.local/lib/python3.10/site-packages/sklearn/base.py:493 UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names

/home/jtsarmento/.local/lib/python3.10/site-packages/sklearn/base.py:493 UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names

/home/jtsarmento/.local/lib/python3.10/site-packages/sklearn/base.py:493 UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names

/home/jtsarmento/.local/lib/python3.10/site-packages/sklearn/base.py:493 UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names

/home/jtsarmento/.local/lib/python3.10/site-packages/sklearn/base.py:493 UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names

/home/jtsarmento/.local/lib/python3.10/site-packages/sklearn/base.py:493 UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names

/home/jtsarmento/.local/lib/python3.10/site-packages/sklearn/base.py:493 UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names

,symbol,last_date,next_date,predicted_close,last_close,predicted_change,predicted_pct
0,AAPL,2025-04-25,2025-04-28,209.706492,209.279999,0.426494,0.203791
1,AMZN,2025-04-25,2025-04-28,189.139203,188.990005,0.149198,0.078945
2,CELH,2025-04-25,2025-04-28,36.826101,36.740002,0.086099,0.234348
3,EU,2025-04-25,2025-04-28,1.586000,1.600000,-0.014000,-0.875001
4,GOOGL,2025-04-25,2025-04-28,161.555554,161.960007,-0.404453,-0.249724
5,META,2025-04-25,2025-04-28,541.483181,547.270020,-5.786838,-1.057401
6,MSFT,2025-04-25,2025-04-28,389.700004,391.850006,-2.150002,-0.548680
